In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

# --- PATH AYARLARI (Local Yapıya Uygun) ---
# Eğer Colab'da çalışacaksan bu kısmı değiştirip dosyaları yanına yüklemen yeterli.
# Localde: notebooks klasörünün bir üstüne çıkıp input'a gidiyoruz.
current_dir = os.getcwd()
# Eğer notebook'tan çalışıyorsan ../input, script ise os.path mantığı
INPUT_DIR = os.path.join(os.path.dirname(current_dir), "input") 
OUTPUT_DIR = os.path.join(os.path.dirname(current_dir), "output", "submissions")

# Klasör yoksa oluştur
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- DONANIM AYARLARI ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 64
EPOCHS = 15
LR = 0.001
NUM_CLASSES = 300

print(f"Cihaz: {DEVICE}")
print(f"Veri Yolu: {INPUT_DIR}")

# 1. Dosyaları Yükle
print("Veriler belleğe yükleniyor...")
try:
    train_emb = np.load(os.path.join(INPUT_DIR, "train_embeddings.npy"), allow_pickle=True)
    train_ids = np.load(os.path.join(INPUT_DIR, "train_ids.npy"), allow_pickle=True)
    test_emb = np.load(os.path.join(INPUT_DIR, "test_embeddings.npy"), allow_pickle=True)
    test_ids = np.load(os.path.join(INPUT_DIR, "test_ids.npy"), allow_pickle=True)
    terms_df = pd.read_csv(os.path.join(INPUT_DIR, "train_terms.tsv"), sep="\t")
except FileNotFoundError as e:
    print(f"HATA: Dosyalar bulunamadı! Lütfen input klasörünü kontrol edin.\n{e}")
    raise

# Tensorlara çevir (Test verisi için)
X_test_tensor = torch.tensor(test_emb, dtype=torch.float32).to(DEVICE)

# 2. Model Mimarisi (MLP)
class SimpleProteinModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SimpleProteinModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, 512)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.layer2 = nn.Linear(512, num_classes)
    
    def forward(self, x):
        return self.layer2(self.dropout(self.relu(self.layer1(x))))

# 3. Ana Döngü (MF, BP, CC)
aspects = {'F': 'Molecular Function', 'P': 'Biological Process', 'C': 'Cellular Component'}
submission_data = []

print("\n--- EĞİTİM ve TAHMİN SÜRECİ BAŞLIYOR ---")

for aspect_code, aspect_name in aspects.items():
    print(f"\n>>> İşleniyor: {aspect_name} ({aspect_code})")
    
    # Hedefleri Hazırla
    df_aspect = terms_df[terms_df['aspect'] == aspect_code]
    top_terms = df_aspect['term'].value_counts().head(NUM_CLASSES).index.tolist()
    
    term_to_idx = {term: i for i, term in enumerate(top_terms)}
    id_map = {pid: i for i, pid in enumerate(train_ids)}
    
    labels = np.zeros((len(train_ids), NUM_CLASSES), dtype=np.float32)
    valid_rows = df_aspect[df_aspect['term'].isin(top_terms) & df_aspect['EntryID'].isin(train_ids)]
    
    for _, row in valid_rows.iterrows():
        if row['EntryID'] in id_map:
            labels[id_map[row['EntryID']], term_to_idx[row['term']]] = 1.0
            
    # Eğitim Verisi
    X_train = torch.tensor(train_emb, dtype=torch.float32)
    y_train = torch.tensor(labels, dtype=torch.float32)
    
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    # Model Eğitimi
    model = SimpleProteinModel(input_dim=320, num_classes=NUM_CLASSES).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LR)
    criterion = nn.BCEWithLogitsLoss()
    
    model.train()
    for epoch in range(EPOCHS):
        total_loss = 0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(DEVICE), batch_y.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
    # Test Tahmini
    print(f"   {aspect_name} tahmin ediliyor...")
    model.eval()
    with torch.no_grad():
        test_logits = model(X_test_tensor)
        test_probs = torch.sigmoid(test_logits).cpu().numpy()
        
    for i in range(len(test_ids)):
        pid = test_ids[i]
        for j in range(NUM_CLASSES):
            score = test_probs[i, j]
            if score > 0.01: 
                submission_data.append([pid, top_terms[j], round(score, 3)])

# 4. Kaydet
print("\n--- KAYDEDİLİYOR ---")
df_sub = pd.DataFrame(submission_data, columns=['ProteinID', 'GO_Term', 'Score'])
output_path = os.path.join(OUTPUT_DIR, "submission_v1_baseline.tsv")

df_sub.to_csv(output_path, sep="\t", header=False, index=False)
print(f"Dosya oluşturuldu: {output_path}")